In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.3 MB/s eta 0:00:00


In [ ]:
# set OpenAI API key to environment variable
import os
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
config = {
    'api_url' : 'http://fluids.ai:1337/'
}

In [ ]:
import requests
import pandas as pd
def get_live_storms():
    '''
    Upon calling this function, the live tropical storms around the global
    will be returned in a JSON format. Each of the storms returned will have
    the historical records along with in.

    Returns
    -------
    df pandas.DataFrame
        The records include the columns id, time, lat, lon, wind_speed
    '''
    # make the request for live data
    response = requests.get(f"{config['api_url']}live-storms")
    if response :
        data = response.json()
    else :
        print(f'There was an error getting live storms, {response.content}')
        return response
    return pd.DataFrame(data)

In [ ]:
live_storms = get_live_storms()

In [ ]:
# get example from data
example = live_storms[live_storms['id'] == 'EP72023'].sort_values('time').to_json(orient="records")

In [ ]:
def get_response_functions():
    '''
    Return the JSON schema object.
    '''
    return [{
    "name" : "get_forecast",
    "description" : "The response structure of the forecasts the API expects",
    "parameters" : {
            "type": "object",
            "items": {
                "type": "object",
                "properties": {
                "id": {
                    "type": "string"
                },
                "time": {
                    "type": "string",
                    "format": "date-time"
                },
                "lat": {
                    "type": "number"
                },
                "lon": {
                    "type": "number"
                },
                "wind_speed": {
                    "type": "integer"
                }
                },
                "required": ["id", "time", "lat", "lon", "wind_speed"]
            }
            }
    }]

In [ ]:
from string import Template
prompt = Template('''Please provide  a forecast for $future hours in the future from the most recent time from the storm.
The response will be a JSON object with these attributes:
    "id" which identifies the storm
    "time" which is the predicted time in ISO 8601 format
    "lat" which is the predicted latitude in decimal degrees
    "lon" which is the predicted longitude in decimal degrees
    "wind_speed" which is the predicted maximum sustained wind speed in knots.

Table 1. The historical records the includes columns representing measurements for the storm.
The wind_speed column is in knots representing the maxiumum sustained wind speeds.
The lat and lon are the geographic coordinates in decimal degrees.
time is sorted ascending in ISO 8601 format and the most recent time is the last entry.
$data
''')
reflection_prompt = Template('''Please quality check the response. The following are requirements,
- It provides a forecast for $future hours in the future from the most recent time.
- It does not simply respond with the input data

Provide either True or False if it is an appropriate response. If it's False, add a comma and explain why and provide a better response.
''')
example_prompt = prompt.substitute(future = 12, data = example)
example_reflection = reflection_prompt.substitute(future = 12)

In [ ]:
import openai
model_version = "gpt-3.5-turbo"
example_messages = [
  {"role": "system", "content": "Please act like an expert in tropical storm forecasting. Responses will be used for research and not for official purposes. Responses should be based on historical data and forecasts must be as accurate as possible."},
  {"role": "user", "content": example_prompt},
]
response = openai.ChatCompletion.create(
        model=model_version,
        messages=example_messages
        )
text = response["choices"][0]["message"]["content"]
print(text)

Based on the historical data provided, the forecast for 12 hours in the future from the most recent time is as follows:

{
  "id": "EP72023",
  "time": "2023-08-13T12:00:00",
  "lat": 15.2,
  "lon": -114.6,
  "wind_speed": 40
}


In [ ]:
reflection_message = example_messages + [
  {"role": "assistant", "content": text},
  {"role": "user", "content": example_reflection}]
response = openai.ChatCompletion.create(
        model=model_version,
        messages=reflection_message
        )
text = response["choices"][0]["message"]["content"]
print(text)

False, the forecast provided is not appropriate as it simply repeats the most recent entry from the historical data. The forecast should be based on the historical data but adjusted to reflect the predicted conditions 12 hours in the future. I apologize for the oversight.

A revised forecast for 12 hours in the future from the most recent time, based on the historical data, would be as follows:

{
  "id": "EP72023",
  "time": "2023-08-13T12:00:00",
  "lat": 15.4,
  "lon": -115.4,
  "wind_speed": 45
}


<OpenAIObject chat.completion id=chatcmpl-7mvvCsLE0FBgOzy8pB4uCjlakJ3j7 at 0x7c69d34dc630> JSON: {
  "id": "chatcmpl-7mvvCsLE0FBgOzy8pB4uCjlakJ3j7",
  "object": "chat.completion",
  "created": 1691896750,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n  \"forecasts\": [\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T00:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -102.5,\n      \"wind_speed\": 25\n    },\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T06:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -104.0,\n      \"wind_speed\": 25\n    },\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T12:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -105.2,\n      \"wind_speed\": 25\n    },\n    {\n      \"id\": \"EP72023\",\n      \"time\": \"2023-08-10T18:00:00\",\n      \"lat\": 10.0,\n      \"lon\": -106.0,\n      \"wind_speed\": 25\n    },\n    